In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from six import StringIO
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import math
from ml import get_X_y, get_label
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

from process_evosuite_data import ProcessEvoSuiteData
from feature_selection_extraction import select_features

data = ProcessEvoSuiteData()

labels_dict_gl = dict()
labels_dict_gl[60] = {'branch_60':0, 'default_60':1, 'weak_60':2}
labels_dict_gl[180] = {'branch_180':0, 'default_180':1, 'weak_180':2}
labels_dict_gl[300] = {'branch_300':0, 'default_300':1, 'weak_300':2}

how_many = 0


In [2]:
def run_KFold(search_budgets=[60,180,300], columns_to_group=['TARGET_CLASS', 'configuration_id', 'project.id'], score_metric='BranchCoverage', score_metric_filename='res_data/results-'):
    """
    Run KFold cross validation using X, y and the chosen models.
    """

    f1s = dict()
    for search_budget in search_budgets:

        # Get samples and their labels
        X, y, features = get_X_y(search_budget, columns_to_group, score_metric, score_metric_filename, labels_dict=None)
        X = np.array(X)
        y = np.array(y)

        # Apply dataset balancing techniques
        over_sampler = RandomOverSampler(random_state=42)
        X_res, y_res = over_sampler.fit_resample(X, y)
        # under_sampler = RandomUnderSampler(random_state=42)
        # X_res, y_res = under_sampler.fit_resample(X, y)

        kf = KFold(n_splits=10, random_state=42, shuffle=True)
        f1s_svc = []
        f1s_dt = []
        f1s_rf = []
        f1s_lr = []

        for train_index, test_index in kf.split(X_res):
            # Split the data
            X_train, X_test = X_res[train_index], X_res[test_index]
            y_train, y_test = y_res[train_index], y_res[test_index]            

            # Feature selection
            X_train, X_test, selected_features = select_features(X_train, X_test, features)
            
            # Train SVC
            clf = SVC()
            clf.set_params(kernel='rbf').fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            f1s_svc.append(f1_score(y_pred, y_test, average='weighted'))
            
            # Train Decision Tree
            decision_tree = tree.DecisionTreeClassifier(random_state=456).fit(X_train, y_train)
            y_pred = decision_tree.predict(X_test)
            f1s_dt.append(f1_score(y_pred, y_test, average='weighted'))
            # str_preds = [str(x) for x in y_pred]
            # fig = plt.figure(figsize=(25,20))
            # _ = tree.plot_tree(decision_tree, 
            #        feature_names=features,  
            #        class_names=str_preds,
            #        filled=True)
            # # tree.plot_tree(decision_tree)
            # fig.savefig("decistion_tree.png")
            # text_representation = tree.export_text(decision_tree)
            # with open("decistion_tree.log", "w") as fout:
            #     fout.write(text_representation)


            dot_data = StringIO()
            export_graphviz(decision_tree, out_file=dot_data,  
                            filled=True, rounded=True,
                            special_characters=True, feature_names = selected_features,class_names=['0','1','2'])
            graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
            graph.write_png('dec_tree' + score_metric + '_' + str(search_budget) + '.png')
            Image(graph.create_png())

            # Train Random Forest classifier
            random_forest = RandomForestClassifier(n_estimators = 100).fit(X_train, y_train)
            y_pred = random_forest.predict(X_test)
            f1s_rf.append(f1_score(y_pred, y_test, average='weighted'))

            # Train a Logistic Regression classifier
            logistic_regression = LogisticRegression(random_state=0, max_iter=300).fit(X_train, y_train)
            y_pred = logistic_regression.predict(X_test)
            f1s_lr.append(f1_score(y_pred, y_test, average='weighted'))

        f1s[search_budget] = (f1s_dt, f1s_svc, f1s_rf, f1s_lr)

    return f1s


In [3]:
f1s_coverage = run_KFold(search_budgets=[60, 180, 300], columns_to_group=['TARGET_CLASS', 'configuration_id', 'project.id'], score_metric='BranchCoverage', score_metric_filename='res_data/results-')
print( 'DT: ' + str(np.average(f1s_coverage[60][0])) + ' SVC: ' + str(np.average(f1s_coverage[60][1])) + ' RF: ' + str(np.average(f1s_coverage[60][2])) + ' LR: ' + str(np.average(f1s_coverage[60][3])))
print( 'DT: ' + str(np.average(f1s_coverage[180][0])) + ' SVC: ' + str(np.average(f1s_coverage[180][1])) + ' RF: ' + str(np.average(f1s_coverage[180][2])) + ' LR: ' + str(np.average(f1s_coverage[180][3])))
print( 'DT: ' + str(np.average(f1s_coverage[300][0])) + ' SVC: ' + str(np.average(f1s_coverage[300][1])) + ' RF: ' + str(np.average(f1s_coverage[300][2])) + ' LR: ' + str(np.average(f1s_coverage[300][3])))

f1s_mutation_score = run_KFold(search_budgets=[60], columns_to_group=['class', 'configuration', 'project'], score_metric='mutation_score_percent', score_metric_filename='res_data/mutation_scores.csv')
print( 'DT_mutation: ' + str(np.average(f1s_mutation_score[60][0])) + ' SVC_mutation: ' + str(np.average(f1s_mutation_score[60][1])) + ' RF: ' + str(np.average(f1s_mutation_score[60][2])) + ' LR: ' + str(np.average(f1s_mutation_score[60][3])))


/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3159: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3159: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/daniela/.local/lib/python3.8/site-packages/sklearn/linea

DT: 0.7353102572345255 SVC: 0.4285095340338324 RF: 0.729356221704294 LR: 0.4305325293364183
DT: 0.8544118132372146 SVC: 0.38691784335762364 RF: 0.859589427573035 LR: 0.36557233697330965
DT: 0.7887685714552475 SVC: 0.5374077567240136 RF: 0.7873479289691481 LR: 0.4692732504915648


/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3159: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/daniela/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3159: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/home/daniela/.local/lib/python3.8/site-packages/sklearn/linea

DT_mutation: 0.7300933573235338 SVC_mutation: 0.42906032253549276 RF: 0.7427975084943486 LR: 0.4604009777743241


/home/daniela/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
